In [1]:
# Get imports
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt

In [2]:
# import helper functions
import camera_calibrator
import binary_gradient
import perspective_transform
import measure_curvature
import original_perspective

In [3]:
test4 = ('./test_images/test4.jpg')

In [4]:
def pipeline(image):
    
    # Directory of images for calibration 
    image_directory = glob.glob('camera_cal/calibration*.jpg')
      
    # Get matrix from calibration
    mtx, dist = camera_calibrator.calibration(image_directory)
    
    # Undistored image 
    undistored_image = cv2.undistort(image, mtx, dist, None, mtx)
    
    # set thresholds for edge detection 
    sobel_threshold = (170, 255)
    sobelx_threshold = (20, 100)
    
    # Binary Image 
    color_binary, binary_image = binary_gradient.binary_image(undistored_image, sobel_threshold, sobelx_threshold)
    
    # set top and bottom margins
    top_margin = 94
    bottom_margin = 451
    
    # Perspective Transform
    perspective_transformed_image = perspective_transform.get_transformed_perspective(binary_image, top_margin, bottom_margin)
    
    # set margins, sliding windows, pixels to recenter window
    margin = 99
    nwindows = 9
    minpixels = 50

    # convert back to original image space 
    color_warp = measure_curvature.get_colored_area(perspective_transformed_image, margin, nwindows, minpixels)
    warp_image = original_perspective.get_original_perspective(color_warp, top_margin, bottom_margin)

    result = cv2.addWeighted(image, 1, warp_image, 0.3, 0)
    
    return result

In [6]:
# Import everything needed to edit/save/watch video clips
# from moviepy.editor import VideoFileClip
# from IPython.display import HTML

In [7]:
#clip1 = VideoFileClip('project_video.mp4')
#clip = clip1.fl_image(pipeline)
#%time clip.write_videofile('project_video_processed_copy.mp4', audio=False)